# Introduction

This code reads the scio and/or saves to a json file, then tries to decode it.

In [ ]:
# To read settings file
import json
# Parst the command line arguments
import argparse

# Logging/output format setup
import logging
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
#logging.basicConfig(format='[%(asctime)s] %(levelname)8s %(module)15s: %(message)s')
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

# The SCIO library
#import sciolib.sciolib as scio

In [ ]:
def main_fct(calibrate, infile, outfile, calfile, protocol, raw):
    if(calibrate): # DEBUG!!!
        log.debug("We were told to calibrate")
    log.info("Input/output")
        
    # Find the home folder (temporary)
    from pathlib import Path
    json_dir = str(Path.home())
    
    # Check if we need to scan
    if(infile is not None):
        # Input comes from a file
        log.info("--> Input through:    " + json_dir + "/" + infile)
        log.info("--> Output file name: " + "N/A")
        # Read the raw json file
        json_df = scio.raw_read(json_dir + "/" + infile)
        scan_raw_df = json_df[7:10] # Element 10 is not included...
    else:
        # We have to scan (Only USB currently works)
        log.info("--> Input through:    " + protocol)
        log.info("--> Output file name: " + outfile)
        
        # Try to find the device
        scio_device = scio.find_scio_dev()
    
        # https://makersportal.com/blog/2018/2/25/python-datalogger-reading-the-serial-output-from-arduino-to-analyze-data-using-pyserial
        try: # Make sure to close the serial port if it was still open
            ser.close()
        except:
            pass
    
        # Read temperature before scanning
        temp_before_df = scio.read_data(scio_device, 4) # 4 = read temperature
        log.info("CMOS T: {:.3f}".format(temp_before_df[0])) # cmosTemperature, chipTemperature, objectTemperature
        log.info("Chip T: {:.3f}".format(temp_before_df[1]))
        log.info("Obj. T: {:.3f}".format(temp_before_df[2]))
    
        # Scan and decode
        scan_raw_df = scio.read_data(scio_device, 2) # 2 = read data
    
        # Read temperature after scanning
        temp_after_df = scio.read_data(scio_device, 4)
    
        # Path to output file within home directory
        if(outfile == "scio-scan"):
            # The default file name was given!
            log.info("WARNING! The default file was used. Any existing previous file will be overwritten!")
            
        # Save the file
        log.debug("Writing raw scan to: " + json_dir + "/" + outfile + ".json")
        # Saves the raw data. In the end, I'll want to save the scan as a CSV
        scio.raw_write(temp_before_df, temp_after_df, scan_raw_df, json_dir + "/" + outfile + ".json")
    
    #print(scio.read_data(scio_device, 1)) # Read device ID
    #print(scio.read_data(scio_device, -124)) # Read BLE ID
    
    if(calfile is not None):
        # read calibration file
        log.info("--> Calibration file name: " + "N/A")
        # Read the raw json file
        json_df = scio.raw_read(json_dir + "/" + calfile)
        scan_cal_df = json_df[7:10] # Element 10 is not included...
    
    log.info("Trying to decode data")
    #scan_df = scio.decode_data(scan_raw_df) # DOES NOT YET WORK
    scan_df = scio.decode_data2(scan_raw_df, scan_cal_df)
    #print(scan_df) #DEBUG
    #log.debug("Number of scans in data:  " + str(len(scan_df)))
    #log.debug("Number of variables/scan: " + str(len(scan_df[0])))
    print("")

In [ ]:
main_fct(calibrate, infile, outfile, calfile, protocol, raw)